In [1]:
import liana as li
import scanpy as sc
from anndata import AnnData
import pandas as pd
from liana.mt import rank_aggregate
from liana.method import singlecellsignalr, connectome, cellphonedb, natmi, logfc, cellchat, geometric_mean
from pycirclize import Circos

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# loading original anndata
pvat_24weeks_hf_female = sc.read_h5ad('../../exported_data/pvat_24weeks_hf_female.h5ad')
pvat_24weeks_hf_female

AnnData object with n_obs × n_vars = 8521 × 20743
    obs: 'sample_id', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'n_genes', 'scDblFinder_score', 'scDblFinder_class', 'Sample', 'tissue', 'diet', 'time', 'sex', 'batch', 'sample_type', 'rat_id', 'weight_3W', 'weight_10W', 'weight_17W', 'weight_24W', 'MAP_7W', 'systolic_bp_7W', 'MAP_9W', 'systolic_bp_9W', 'MAP_16W', 'systolic_bp_16W', 'MAP_24W', 'systolic_bp_24W', 'doublet', 'celltype', 'celltype_broad'
    var: 'gene_ids', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'
    uns: 'cellchat_interaction_table_count', 'cellchat_res', 'cellchat_res_filtered', 

In [3]:
pvat_24weeks_hf_female.obs['celltype_broad'].unique().tolist()

['Endothelial_Cells',
 'Adipocytes',
 'Immune_Cells',
 'Fibroblasts',
 'Mesothelial_Cells',
 'Pericytes',
 'SMCs',
 'Neuronal_Cells']

In [4]:
# List available methods in lliana framework
li.mt.show_methods()

,Method Name,Magnitude Score,Specificity Score,Reference
0,CellPhoneDB,lr_means,cellphone_pvals,"Efremova, M., Vento-Tormo, M., Teichmann, S.A...."
0,Connectome,expr_prod,scaled_weight,"Raredon, M.S.B., Yang, J., Garritano, J., Wang..."
0,log2FC,None,lr_logfc,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,NATMI,expr_prod,spec_weight,"Hou, R., Denisenko, E., Ong, H.T., Ramilowski,..."
0,SingleCellSignalR,lrscore,None,"Cabello-Aguilar, S., Alame, M., Kon-Sun-Tack, ..."
0,CellChat,lr_probs,cellchat_pvals,"Jin, S., Guerrero-Juarez, C.F., Zhang, L., Cha..."
0,Rank_Aggregate,magnitude_rank,specificity_rank,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,Geometric Mean,lr_gmeans,gmean_pvals,CellPhoneDBv2's permutation approach applied t...


In [5]:
pvat_24weeks_hf_female.raw = pvat_24weeks_hf_female
pvat_24weeks_hf_female.raw.X

<8521x20743 sparse matrix of type '<class 'numpy.float64'>'
	with 12368377 stored elements in Compressed Sparse Row format>

In [6]:
# Run CellPhoneDB analysis on PVAT data
cellphonedb(pvat_24weeks_hf_female, groupby='celltype_broad', expr_prop=0.1, resource_name='mouseconsensus', verbose=True, key_added='cellphonedb_res')

Using `.raw`!
/Users/pandavis/miniconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
1477 features of mat are empty, they will be removed.
Make sure that normalized counts are passed!
/Users/pandavis/miniconda3/lib/python3.11/site-packages/liana/method/_pipe_utils/_pre.py:148: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
['Cyp2c6_v1-ps2'] contain `_`. Consider replacing those!
Using resource `mouseconsensus`.
0.20 of entities in the resource are missing from the data.


Generating ligand-receptor stats for 8521 samples and 19266 features


100%|███████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 273.12it/s]


In [7]:
ligand_receptor_pvat = pvat_24weeks_hf_female.uns['cellphonedb_res']
ligand_receptor_pvat.head()

,ligand,ligand_complex,ligand_means,ligand_props,receptor,receptor_complex,receptor_means,receptor_props,source,target,lr_means,cellphone_pvals
2176,Slit3,Slit3,27.483875,1.000000,Robo4,Robo4,0.413083,0.293459,SMCs,Endothelial_Cells,13.948480,0.0
46,App,App,1.808728,0.574963,Lrp6,Lrp6,17.855547,0.964365,Adipocytes,Adipocytes,9.832137,0.0
818,App,App,1.537366,0.633452,Lrp6,Lrp6,17.855547,0.964365,Pericytes,Adipocytes,9.696457,0.0
302,App,App,1.238891,0.552778,Lrp6,Lrp6,17.855547,0.964365,Fibroblasts,Adipocytes,9.547219,0.0
156,App,App,1.117851,0.520330,Lrp6,Lrp6,17.855547,0.964365,Endothelial_Cells,Adipocytes,9.486699,0.0


In [8]:
#ligand_receptor_string = (ligand_receptor_pvat.loc[:,'ligand_complex'].str.contains('Col')) & (ligand_receptor_pvat.loc[:,'receptor_complex'].str.contains('Itg'))
cell_source_string = ligand_receptor_pvat.loc[:,'source'] == 'Fibroblasts'
cell_target_string = ligand_receptor_pvat.loc[:,'target'] == 'Adipocytes'
cell_pvals = ligand_receptor_pvat.loc[:,'cellphone_pvals'] <= 0.05

ligand_receptor_pvat[ligand_receptor_string & cell_source_string & cell_target_string & cell_pvals]

,ligand,ligand_complex,ligand_means,ligand_props,receptor,receptor_complex,receptor_means,receptor_props,source,target,lr_means,cellphone_pvals
250,Col4a1,Col4a1,3.683332,0.609722,Itgb1,Itga1_Itgb1,0.443682,0.295606,Fibroblasts,Adipocytes,2.063507,0.000
279,Col5a3,Col5a3,2.748613,0.670833,Itgb1,Itga1_Itgb1,0.443682,0.295606,Fibroblasts,Adipocytes,1.596148,0.000
285,Col3a1,Col3a1,2.676389,0.716667,Itgb1,Itga1_Itgb1,0.443682,0.295606,Fibroblasts,Adipocytes,1.560036,0.000
256,Col4a2,Col4a2,2.555556,0.550000,Itgb1,Itga1_Itgb1,0.443682,0.295606,Fibroblasts,Adipocytes,1.499619,0.011
333,Col4a2,Col4a2,2.555556,0.550000,Itgb5,Itgb5,0.300591,0.205133,Fibroblasts,Adipocytes,1.428074,0.002
260,Col6a2,Col6a2,1.938893,0.650000,Itgb1,Itga1_Itgb1,0.443682,0.295606,Fibroblasts,Adipocytes,1.191288,0.000
281,Col5a2,Col5a2,1.779171,0.666667,Itgb1,Itga1_Itgb1,0.443682,0.295606,Fibroblasts,Adipocytes,1.111426,0.000
244,Col6a6,Col6a6,1.554170,0.462500,Itgb1,Itga1_Itgb1,0.443682,0.295606,Fibroblasts,Adipocytes,0.998926,0.000
258,Col6a1,Col6a1,1.344447,0.597222,Itgb1,Itga1_Itgb1,0.443682,0.295606,Fibroblasts,Adipocytes,0.894064,0.000
262,Col6a3,Col6a3,1.309724,0.476389,Itgb1,Itga1_Itgb1,0.443682,0.295606,Fibroblasts,Adipocytes,0.876703,0.000


In [9]:
ligand_receptor_pvat.to_csv("pvat_24weeks_hf_female_cellphonedb.csv")
pvat_24weeks_hf_female.write("../../exported_data/pvat_24weeks_hf_female.h5ad")